# EfficientNetB7 - Daisy Wang

## pip3 install requirements

In [3]:
!pip3 install -r ../../requirements.txt
!pip3 install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 20.6 MB/s eta 0:00:0000:0100:01


## Import packages that we need

In [25]:
# import tensorflow/keras stuff
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split

# import data processing/visualisation things
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
%matplotlib inline

# importing misc stuff
import os
import cv2

## download the data (if needed)

In [26]:
# commented out so that i don't accidentally run this again when speedrunning notebook
#!kaggle datasets download -d andrewmvd/lung-and-colon-cancer-histopathological-images

## import data???

### function to load images + downgrade data

In [14]:
# function to load images
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # resize so my cpu 
            img = cv2.resize(img, (80,80))
            images.append(img)
            
    return np.array(images)

### load images with the correct names

Note: need to modify the path based on where you're running this from

In [15]:
benign = load_images('lung_colon_image_set/lung_image_sets/lung_n')
mal_aca = load_images('lung_colon_image_set/lung_image_sets/lung_aca')
mal_scc = load_images('lung_colon_image_set/lung_image_sets/lung_scc')

### make sure that everything is correct

In [17]:
print(f"Number of images for every class: BENIGN {benign.shape[0]}, ADENOCARCINOMAS {mal_aca.shape[0]}, \
SQUAMOS CELL CARCINOMAS {mal_scc.shape[0]}.")
print(f"Images shape: {benign[0].shape}.")

Number of images for every class: BENIGN 5000, ADENOCARCINOMAS 5000, SQUAMOS CELL CARCINOMAS 5000.
Images shape: (80, 80, 3).


### get samples and labels!

In [18]:
samples = np.concatenate((benign, mal_aca, mal_scc))
labels = np.array(benign.shape[0] * [0] + mal_aca.shape[0] * [1] + mal_scc.shape[0] * [2])

### check that everything is correct

In [22]:
print(f"Samples shape check: {samples.shape}.")
print(f"Labels shape check: {labels.shape}.")

Samples shape check: (15000, 80, 80, 3).
Labels shape check: (15000,).


In [23]:
indices = np.arange(samples.shape[0])
np.random.shuffle(indices)

samples = samples[indices]
labels = labels[indices]

# normalize pictures
samples = samples.astype('float32') / 255

## train test split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(samples, labels, test_size = 0.2)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

## MODEL TIME RAAAAAH

### Setup the model using Keras

In [29]:
model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(80, 80, 3))

258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
